In [1]:
!pip install sentence_transformers
!pip install konlpy
!pip install customized_konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 4.9 MB 59.4 MB/s 
     |████████████████████████████████| 1.3 MB 37.3 MB/s 
     |████████████████████████████████| 163 kB 67.1 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=de558fb14f05b99b9f157221f3c1cece69a733a812c025153468b1791c5fc0e4
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.0 MB/s 
     |████████████████████████████████| 453 kB 51.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.de

In [2]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from ckonlpy.tag import Twitter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt
import re

In [3]:
doc = """
1절 분석기획 방향성 도출

​

분석기획이란

실제 분석 수행에 앞서 분석을 수행할 과제를 정의하고, 의도했던 결과를 도출할 수 있도록

적절하게 관리할 수 있는 방안을 사전에 계획하는 일련의 작업

분석과제나 프로젝트 직접 수행하는 것은 아님

목표(What) 달성을 위해(Why) 어떤 데이터를 가지고 어떤 방식으로(How) 수행할 지 계획 수립

​

분석 기획에서의 데이터 사이언티스트 역량

해당 문제 영역에 대한 전문성 역량 및 수학/통계학적 지식 뿐 아니라

분석의 도구인 데이터 및 프로그래밍 기술 역량

균형 잡힌 시각 필요

​

분석 대상과 방법

*분석 기획 위해 필요한 3가지 영역과 분석 대상과 방법에 따른 분석 유형으로 출제 가능성 높음

분석 = 대상(What) 과 방법(How) 에 따라서 4가지로 나누어짐

최적화 : 대상과 방법 둘 다 알고 있음

솔루션 : 대상은 알지만 방법은 모름

통찰 : 대상은 모르지만 방법을 알고 있음

발견 : 대상과 방법 둘 다 모름

​

목표 시점 별 분석 기획 방안

*목표 시점별 분석방향, 목표,유형, 접근방식에 대한 특성을 묻는 문제가 출제 될 수 있음. 내용 이해하기!

목표시점 별 = 과제를 빠르게 해결하는(단기적) '과제 중심적인 접근 방식' 

                       지속적인 분석 내재화를 위한 '장기적인 마스터 플랜 방식'

분석기획에서는 이 두가지를 융합하여 적용하는 것이 중요

- 과제 중심적인 접근 방식 : Speed & Test / Quick & Win / Problem Solving (문제 해결)

- 장기적인 마스터 플랜 방식 ; Accuracy & Deploy / Long Term View / Problem Definition (분석 과제 정의)

​

분석 기획 시 고려사항

1 . 가용 데이터 : 유형에 따라 적용 가능한 솔루션 및 분석방법

2 . 적절한 유즈 케이스 : 분석을 통해 가치가 창출될 수 있는 적절한 활용방안과 유즈케이스 필요

3 . 낮은 실행 장벽 : 장애요소 들에 대한 사전계획 수립 필요

"""

In [4]:
# 불용어데이터
text = pd.read_csv("./stopwords.csv")

In [5]:
text_list = list(np.array(text['불용어'].tolist()))

In [16]:
def keyword_part(doc) :
      twitter = Twitter()
      twitter.add_dictionary(['재표현', '생성','탐색적','파생변수','특이점','탐색적분석','탐색적자료분석','데이터마이닝','최상위','과적합','불명확','접근법','거시적','비교','트레이드오프','산출물','상향식','하향식','연관성','유즈케이스','내재화','해결','고려사항','장기적','단기적','사이언티스트','전처리','이상치','결측치','학습용 데이터','인공지능','텍스트 마이닝','메타버스','키워드','시사점','가속화','선도국','고품질','적극적','투자','불확실성','감소','중장기','장년층','노년층','상대적','최소화','디지털 리터러시 교육','세대별','맞춤형','어디서나','특성','긍정적','시공간'], 'Noun')
      key_part = twitter.pos(doc)
      
      return key_part

In [17]:
okt = Okt()

example = doc
stop_words = text_list

# stop_words = set(stop_words.split(' '))
word_tokens = okt.morphs(example)

result = [word for word in word_tokens if not word in stop_words]

print('불용어 제거 전 :',word_tokens) 
print('불용어 제거 후 :',result)

불용어 제거 전 : ['\n', '1', '절', '분석', '기획', '방향성', '도출', '\n\n', '\u200b', '\n\n', '분석', '기획', '이란', '\n\n', '실제', '분석', '수행', '에', '앞서', '분석', '을', '수행', '할', '과제', '를', '정의', '하고', ',', '의도', '했던', '결과', '를', '도출', '할', '수', '있도록', '\n\n', '적절하게', '관리', '할', '수', '있는', '방안', '을', '사전', '에', '계획', '하는', '일련', '의', '작업', '\n\n', '분석', '과제', '나', '프로젝트', '직접', '수행', '하는', '것', '은', '아님', '\n\n', '목표', '(', 'What', ')', '달성', '을', '위해', '(', 'Why', ')', '어떤', '데이터', '를', '가지', '고', '어떤', '방식', '으로', '(', 'How', ')', '수행', '할', '지', '계획', '수립', '\n\n', '\u200b', '\n\n', '분석', '기획', '에서의', '데이터', '사이', '언', '티스', '트', '역량', '\n\n', '해당', '문제', '영역', '에', '대한', '전문성', '역량', '및', '수학', '/', '통계학', '적', '지식', '뿐', '아니라', '\n\n', '분석', '의', '도구', '인', '데이터', '및', '프로그래밍', '기술', '역량', '\n\n', '균형', '잡힌', '시각', '필요', '\n\n', '\u200b', '\n\n', '분석', '대상', '과', '방법', '\n\n', '*', '분석', '기획', '위해', '필요한', '3', '가지', '영역', '과', '분석', '대상', '과', '방법', '에', '따른', '분석', '유형', '으로', '출제', '가능성', '높음', '\n\n

In [18]:
key_part = keyword_part(doc)

tokenized_nouns = ','.join([word[0] for word in key_part if word[1] == 'Noun'])

# print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

명사 추출 : 절,분석,기획,방향성,도출,분석,기획,실제,분석,수행,앞서,분석,수행,과제,정의,의도,결과,도출,수,도록,적,절하,관리,수,방안,사전,계획,일련,작업,분석,과제,프로젝트,직접,수행,것,목표,달성,위해,데이터,가지,방식,수행,계획,수립,분석,기획,데이터,사이언티스트,역량,해당,문제,영역,대한,전문성,역량,및,수학,통계학,적,지식,뿐,분석,도구,데이터,및,프로그래밍,기술,역량,균형,시각,필요,분석,대상,방법,분석,기획,위해,필요,가지,영역,분석,대상,방법,분석,유형,출제,가능성,분석,대상,과,방법,따라서,가지,최적화,대상,방법,둘,알,솔루션,대상,알,방법,모름,통찰,대상,방법,알,발견,대상,방법,둘,모름,목표,시점,별,분석,기획,방안,목표,시점,별,분석,방향,목표,유형,접근,방식,대한,특성,문제,출제,수,내용,이해,목표,시점,별,과제,해,결하,단기적,과제,중심,적,접근,방식,지속,적,분석,내재화,위,장기적,마스터,플랜,방식,분석,기획,이,두,가지,융합,적용,것,중요,과제,중심,적,접근,방식,문제,해결,장기적,마스터,플랜,방식,분석,과제,정의,분석,기획,시,고려사항,가용,데이터,유형,적용,가능,솔루션,및,분석,방법,적절,유즈,케이스,분석,통해,가치,창,수,적절,활용,방안,유즈케이스,필요,실행,장벽,장애,요소,대한,사전,계획,수립,필요


In [19]:
# 유사도와 관련없이 뽑은 키워드들
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 319
trigram 다섯개만 출력 : ['가능 솔루션' '가능 솔루션 분석' '가능성 분석' '가능성 분석 대상' '가용 데이터']


In [20]:
# 키워드들을 수치화
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [21]:
# 문서와 유사한 키워드 뽑기
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['분석 도구 데이터', '분석 도구', '방식 분석 과제', '분석 수행 과제', '도구 데이터 프로그래밍']


In [22]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [23]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['솔루션 분석 방법', '분석 도구 데이터', '분석 도구', '방식 분석 과제', '도구 데이터 프로그래밍']

In [24]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [25]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.3)

['도구 데이터 프로그래밍',
 '필요 분석',
 '플랜 방식 분석',
 '분석 도구',
 '솔루션 분석 방법',
 '전문성 역량 수학',
 '작업 분석',
 '프로그래밍 기술 역량',
 '방식 분석 과제',
 '분석 수행 과제']